# Import Dependencies

Begin by importing the necessary libraries

In [29]:
# Data storage
from google.cloud import bigquery

# Data analysis
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('darkgrid')

# Data Exploration

Explore the StackOverflow dataset from BigQuery's public data archives

In [30]:
# Initialize a client object to project ID
project_id = 'trim-axle-358009'
client = bigquery.Client(project=project_id)

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [31]:
# Print every table ID in the dataset
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

# Question Prompt
Posts (questions & answers) on StackOverflow been declining since 2014.

In [32]:
# Query to select question & answer counts as a function of time
query = """
        WITH q AS (
            SELECT EXTRACT(YEAR FROM creation_date) AS year, COUNT(*) AS num_questions
            FROM `bigquery-public-data.stackoverflow.posts_questions`
            GROUP BY year
        ), 
        a AS (
            SELECT EXTRACT(YEAR FROM creation_date) AS year, COUNT(*) AS num_answers
            FROM `bigquery-public-data.stackoverflow.posts_answers`
            GROUP BY year
        ),
        cte AS (
            SELECT year, num_questions, num_answers,
            FROM q
            JOIN a
                USING(year)
        )
        SELECT 
            c1.year, 
            c1.num_questions,
            ROUND((c1.num_questions - c2.num_questions) / c1.num_questions * 100, 2) AS q_percent_diff,
            c1.num_answers, 
            ROUND((c1.num_answers - c2.num_answers) / c1.num_answers * 100, 2) AS a_percent_diff,
        FROM cte AS c1
        LEFT JOIN cte AS c2
            ON c1.year = c2.year + 1
        ORDER BY c1.year
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [33]:
# drop 2022 data since it's incomplete
df.drop(index=14, inplace=True)

In [34]:
# Make line plot
fig = plt.figure()
ax = df.plot(x='year', y=['num_questions', 'num_answers'], kind='line', linewidth=5, figsize=(15, 15), fontsize=20)

# Customize plot
ax.set_title('Number of Q&A\'s Over Time', fontsize=28)
ax.set_xlabel('')
ax.set_ylabel('Count (Millions)', fontsize=25)
ax.set_yticks(ticks=[0, 500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000], labels=[0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5])
ax.legend(['Questions', 'Answers'], fontsize=25)

# Save plot
plt.savefig('num_qna.png')

In [35]:
# Drop 2008 data since there's no change
df.drop(index=0, inplace=True)

# Make bar plot
fig = plt.figure()
ax = df.plot(x='year', y=['q_percent_diff', 'a_percent_diff'], kind='bar', figsize=(15, 15), fontsize=20)

# Customize plot
ax.set_title('Change in Q&A\'s Over Time', fontsize=28)
ax.set_xlabel('')
ax.set_ylabel('Change (Percent)', fontsize=25)
ax.set_yticks(ticks=[-20, 0, 20, 40, 60, 80], labels=['-20%', '0%', '20%', '40%', '60%', '80%'])
ax.legend(['Questions', 'Answers'], fontsize=25)

# Root Cause Analysis

### 1. Low-hanging fruit 

One possible explanation is that the easier questions have already been asked, e.g. "how to make a plot in Python", "how to read a csv file in Pandas", "how to use SELECT statement in MySQL", etc.

In [36]:
# Query to select most asked about technologies from 2008 to 2014
query = """
        WITH cte_1 AS (
            SELECT
                SPLIT(tags,'|') AS tech_name
            FROM `bigquery-public-data.stackoverflow.posts_questions`
            WHERE EXTRACT(YEAR FROM creation_date) >= 2008 AND EXTRACT(YEAR FROM creation_date) <= 2014
        ),
        cte_2 AS (
            SELECT
              tags,
              COUNT(*) AS num_questions
            FROM cte_1
            CROSS JOIN UNNEST(tech_name) AS tags
            GROUP BY tags
            ORDER BY num_questions DESC
            LIMIT 10
        )
        SELECT 
            tags,
            num_questions,
            ROUND(num_questions / SUM(num_questions) OVER() * 100, 2) AS percent_share
        FROM cte_2
        ORDER BY num_questions DESC
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [37]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = df.tags
sizes = df.percent_share

# "Explode" the 2nd slice (i.e. 'Python')
explode = (0, 0, 0, 0, 0, 0, 0, 0.1, 0, 0) 

# Make pie chart
fig1, ax = plt.subplots(figsize=(15, 17))
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90, textprops={'fontsize': 20})

# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')

# Customize plot
ax.set_title('Top 10 Technologies (2008 - 2014)', fontsize=28)

# Save plot
plt.savefig('tech_distribution_2008.png')

In [38]:
# Query to select most asked about technologies from 2015 to 2022
query = """
        WITH cte_1 AS (
            SELECT
                SPLIT(tags,'|') AS tech_name
            FROM `bigquery-public-data.stackoverflow.posts_questions`
            WHERE EXTRACT(YEAR FROM creation_date) >= 2015 AND EXTRACT(YEAR FROM creation_date) <= 2022
        ),
        cte_2 AS (
            SELECT
              tags,
              COUNT(*) AS num_questions
            FROM cte_1
            CROSS JOIN UNNEST(tech_name) AS tags
            GROUP BY tags
            ORDER BY num_questions DESC
            LIMIT 10
        )
        SELECT 
            tags,
            num_questions,
            ROUND(num_questions / SUM(num_questions) OVER() * 100, 2) AS percent_share
        FROM cte_2
        ORDER BY num_questions DESC
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [39]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = df.tags
sizes = df.percent_share

# "Explode" the 2nd slice (i.e. 'Python')
explode = (0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0) 

# Make pie chart
fig1, ax = plt.subplots(figsize=(15, 17))
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90, textprops={'fontsize': 20})

# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')

# Customize plot
ax.set_title('Top 10 Technologies (2015 - 2022)', fontsize=28)

# Save plot
plt.savefig('tech_distribution_2015.png')

Notice from the above plots that the top 10 most asked-about technologies on StackOverflow has remained largely unchanged pre-2014 vs post-2014. This is evidence for our low-hanging fruit hypothesis, since as time goes on, the pool of remaining questions becomes increasingly more niche. 

### 2. Toxicity

Another explanation may be that users (especially newer ones) feel increasingly intimidated to post questions, out of fear of being "downvoted" or criticized. 

In [40]:
# Query to select number of positively scored questions over time
query = """
        SELECT
          EXTRACT(YEAR FROM creation_date) AS year,
          COUNT(*) AS num_questions,
          ROUND(SUM(IF(score > 0, 1, 0)) / COUNT(*) * 100, 2) AS percent_positive,
          ROUND(SUM(IF(score < 0, 1, 0)) / COUNT(*) * 100, 2) AS percent_negative
        FROM
          `bigquery-public-data.stackoverflow.posts_questions`
        GROUP BY year
        ORDER BY year
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [41]:
# Make bar plot
fig = plt.figure()
ax = df.plot(x='year', y=['percent_positive', 'percent_negative'], kind='bar', figsize=(15, 15), fontsize=20)

# Customize plot
ax.set_title('Question Ratings Over Time', fontsize=28)
ax.set_xlabel('')
ax.set_ylabel('Percentage', fontsize=25)
ax.set_yticks(ticks=[0, 20, 40, 60, 80], labels=['0%', '20%', '40%', '60%', '80%'])
ax.legend(['Positive', 'Negative'], fontsize=25)

# Save plot
plt.savefig('q_ratings.png')

As evidenced by the declining positive ratings and increasing negative ratings towards questions posted to StackOverflow, it's not hard to imagine why new users may feel intimidated out of fear of asking "stupid questions".

# Does it matter? 

Despite the clear downward trend in posts, is there any reason to worry from StackOverflow's perspective? Well, one useful metric to look at to answer this question is new user growth.

In [42]:
# Query to select new_user and inactive_user counts as a function of time
query = """
        WITH cte_1 AS (
            SELECT EXTRACT(YEAR FROM creation_date) AS year, COUNT(*) AS num_new_users
            FROM `bigquery-public-data.stackoverflow.users`
            GROUP BY year
        ),
        cte_2 AS (
            SELECT 
                c1.year,
                c1.num_new_users,
                ROUND((c1.num_new_users - c2.num_new_users) / c1.num_new_users * 100, 2) AS u_percent_diff
            FROM cte_1 AS c1
            LEFT JOIN cte_1 AS c2
                ON c1.year = c2.year + 1
            ORDER BY c1.year
        ),
        cte_3 AS (
            SELECT 
                EXTRACT(YEAR FROM last_access_date) AS last_accessed_year,
                2022 - EXTRACT(YEAR FROM last_access_date) AS years_inactive,
                COUNT(*) AS num_inactive_users
            FROM `bigquery-public-data.stackoverflow.users`
            GROUP BY last_accessed_year, years_inactive
            ORDER BY last_accessed_year
        )
        SELECT c_2.year, c_2.num_new_users, c_3.num_inactive_users
        FROM cte_2 AS c_2
        JOIN cte_3 AS c_3
            ON c_2.year = c_3.last_accessed_year
        ORDER BY c_2.year
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [43]:
# Drop 2022 data since it's incomplete
df.drop(index=14, inplace=True)

In [44]:
# Make line plot
fig = plt.figure()
ax = df.plot(x='year', y=['num_new_users', 'num_inactive_users'], kind='line', linewidth = 5, figsize=(15, 15), fontsize=20)

# Customize plot
ax.set_title('Number of New & Inactive Users Over Time', fontsize=28)
ax.set_xlabel('')
ax.set_ylabel('Count (Millions)', fontsize=25)
ax.set_yticks(ticks=[0, 500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000], labels=[0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5])
ax.legend(['New Users', 'Inactive Users'], fontsize=25)

# Save plot
plt.savefig('user_activity.png')

Notice, the number of new users per year has been steadily climbing, which may provide StackOverflow a sigh of relief. However, somewhat worryingly, the number of inactive users per year has also been increasing, and in 2019, this user-base surpased the aforementioned new-user growth. While this is to be expected, since more users would naturally lead to more inactive users, the relative gap between the two trends should still be addressed.

# What Next?

So, are there any steps StackOverflow can take to accelerate user growth while maintaining user retention?

In [45]:
# Query to select avg_questions_per_year as a function of account creation time
query = """
        WITH cte_1 AS (
            SELECT
                u.id,
                EXTRACT(YEAR FROM u.creation_date) AS year,
                COUNT(*) AS num_questions
            FROM `bigquery-public-data.stackoverflow.users` AS u
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
                ON u.id = q.owner_user_id
            GROUP BY year, u.id
        ),
        cte_2 AS (
            SELECT
                id,
                year,
                num_questions / (2022 - year + 1) AS num_questions_per_year
            FROM cte_1
        )
        SELECT
            year,
            AVG(num_questions_per_year) AS avg_num_questions_per_year
        FROM cte_2
        GROUP BY year
        ORDER BY year
        """

# Set up the query (cancel if greater than 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, then convert the results to a pandas DataFrame
df = query_job.to_dataframe()

# Print the resulting DataFrame
df

In [46]:
# Make bar plot
fig = plt.figure()
ax = df.plot(x='year', y='avg_num_questions_per_year', kind='bar', figsize=(15, 15), fontsize=20)

# Customize plot
ax.set_xlabel('Account Created', fontsize=25)
ax.set_ylabel('Average Number of Questions (per year)', fontsize=25)
ax.legend(['Number of Questions'], fontsize=25)

# Save plot
plt.savefig('avg_questions_vs_age.png')

Interestingly, when looking at the average number of questions per year, with respect to account creation year, we find that users who joined early on during the site's launch in 2008, and users who joined very recently, are the ones posting the most questions! Conversely, users who've been on the site for a few years but are somewhat removed from the early birds are the ones asking the least questions. 

This implies that the early birds may be acting as gatekeepers, with new users initially eager to ask questions, but as time goes on, they become increasingly jaded, and thus less likely to make new posts (some, even abandoning the site altogether). 

Hence, we suggest that StackOverflow police toxicity by introducing an automated filter for clear and obvious non-answers (e.g. "why would you even ask that", "this question has already been asked stupid", etc.). Additionally, StackOverflow could provide users with more incentives for asking and answering questions, such as virtual currency that could be exchanged for merch. 